# 차기작 계획을 위한 분석
## Preprocessing
### 데이터 불러오기

In [20]:
import pandas as pd

data = pd.read_csv("vgames2.csv")
data.head()

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [21]:
data.shape

(16598, 10)

### 결측치 처리

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   16598 non-null  int64  
 1   Name         16598 non-null  object 
 2   Platform     16598 non-null  object 
 3   Year         16327 non-null  float64
 4   Genre        16548 non-null  object 
 5   Publisher    16540 non-null  object 
 6   NA_Sales     16598 non-null  object 
 7   EU_Sales     16598 non-null  object 
 8   JP_Sales     16598 non-null  object 
 9   Other_Sales  16598 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.3+ MB


In [23]:
data.drop("Unnamed: 0", inplace=True, axis=1)

In [24]:
data.isnull().sum()

Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

In [25]:
data.dropna(axis=0, inplace=True)
data.shape

(16241, 9)

### 단위를 백만으로 통일

In [26]:
data.sort_values(by='NA_Sales', ascending=False)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
14347,Animal Planet: Vet Life,Wii,2009.0,Simulation,Activision,90K,0,0,0.01
13798,King of Clubs: Mini Golf,Wii,2008.0,Sports,Oxygen Interactive,90K,0,0,0.01
13057,Mark Davis Pro Bass Challenge,PS2,2002.0,Sports,Natsume,90K,0.07,0,0.02
10650,Mario Kart DS,DS,2005.0,Racing,Nintendo,9.81,7.57,4.13,1.92
6248,Call of Duty: Black Ops,X360,2010.0,Shooter,Activision,9.67,3.73,0.11,1.13
...,...,...,...,...,...,...,...,...,...
8618,Culdcept,3DS,2012.0,Misc,Nintendo,0,0,0.16,0
8611,Tales of VS.,PSP,2009.0,Fighting,Namco Bandai Games,0,0,0.23,0
8606,PachiPara 15: Super Umi Monogatari in Okinawa 2,PS3,2010.0,Misc,Irem Software Engineering,0,0,0.02,0
8604,Card Fight!! Vanguard G: Stride to Victory!!,3DS,2016.0,Misc,FuRyu,0,0,0.04,0


In [27]:
sales = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']

for col in sales: 
    """
    # remove K, M and convert to float
    # seperate K, M as other column (str.extract)
    # fillna(1) for multiply
    # divide num with million
    """
    new = \
        data[col].replace(r'[KM]+$', '', regex=True).astype(float) * \
        data[col].str.extract(r'[\d\.]+([KM]+)', expand=False).\
        fillna(1).\
        replace(['K','M'], [0.001, 1/10**6]).astype(float)                     
    data[col] = new * 1_000_000
    data[col] = data[col].astype(int)
data.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,40000,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,170000,0,0,10000
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,20000,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,40000,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,120000,90000,0,40000


### 연도별 흥행작 확인

In [28]:
data['Year'] = data['Year'].astype(int)
data.sort_values(by='Year', ascending=True)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
9276,Trade & Battle: Card Hero,GB,0,Strategy,Nintendo,0,0,200000,0
16394,Point Blank 3,PS,0,Shooter,Namco Bandai Games,10000,10000,0,0
14666,Strider 2,PS,0,Platform,Virgin Interactive,50000,30000,0,10000
2249,SplashDown,PS2,1,Racing,Atari,340000,270000,0,90000
8681,NCAA Football 2002,PS2,1,Sports,Electronic Arts,770000,600000,0,200000
...,...,...,...,...,...,...,...,...,...
1786,The King of Fighters XIV,PS4,2016,Fighting,Deep Silver,40000,10000,30000,10000
10107,Phantasy Star Online 2 Episode 4: Deluxe Package,PS4,2017,Role-Playing,Sega,0,0,30000,0
5310,Brothers Conflict: Precious Baby,PSV,2017,Action,Idea Factory,0,0,10000,0
15233,Phantasy Star Online 2 Episode 4: Deluxe Package,PSV,2017,Role-Playing,Sega,0,0,10000,0


### 올바르지 않은 연도 데이터 제거

In [29]:
data['Year'].value_counts().sort_index()[:30]

0        3
1        2
2        3
3        7
4        4
5        4
6        7
7        3
8       10
9       10
10       7
11      10
12       2
13       2
14       2
15       5
16       5
86       1
94       1
95       4
96       3
97       2
98       3
1980     8
1981    46
1982    36
1983    17
1984    14
1985    14
1986    20
Name: Year, dtype: int64

In [30]:
df = data[data['Year'] >= 1000].sort_values(by="Year").reset_index(drop=True)
df.head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Boxing,2600,1980,Fighting,Activision,720000,40000,0,10000
1,Freeway,2600,1980,Action,Activision,320000,20000,0,0
2,Defender,2600,1980,Misc,Atari,990000,50000,0,10000
3,Checkers,2600,1980,Misc,Atari,220000,10000,0,0
4,Missile Command,2600,1980,Shooter,Atari,2560000,170000,0,30000
5,Kaboom!,2600,1980,Misc,Activision,1070000,70000,0,10000
6,Bridge,2600,1980,Misc,Activision,250000,20000,0,0
7,Asteroids,2600,1980,Shooter,Atari,4000000,260000,0,50000
8,Crazy Climber,2600,1981,Action,Atari,210000,10000,0,0
9,RealSports Volleyball,2600,1981,Sports,Atari,120000,10000,0,0


- groupby, pivot
- visualization
    - 연도별 흥행작 top5
    - 연도별 흥행 장르 추세
    - 지역별 흥행 장르
    - 플랫폼별 흥행작 (최근 몇년 내)
    - 대세 플랫폼
    - 퍼블리셔는?
    - 집중해야할 지역
    - 최근일수록 가중치
    
최근 인기 많은 작품을 따라한다면?  
어떤 IP가 전통적으로 인기있는지?

### 연도별 출시작 수
각 연도별 출시작 수가 천차만별이다.  
오래된 데이터의 경우 출시작 수도 적고 최근 트렌드와 다를 수 있기 때문에 제외하는게 좋겠다.  
94년 ~ 2016년 데이터가 주로 쓰일 것이다

In [31]:
df.Year.value_counts().sort_index()

1980       8
1981      46
1982      36
1983      17
1984      14
1985      14
1986      20
1987      16
1988      15
1989      17
1990      16
1991      41
1992      43
1993      60
1994     120
1995     215
1996     259
1997     287
1998     374
1999     336
2000     345
2001     478
2002     823
2003     767
2004     736
2005     930
2006     998
2007    1192
2008    1413
2009    1418
2010    1246
2011    1123
2012     650
2013     543
2014     578
2015     606
2016     337
2017       3
2020       1
Name: Year, dtype: int64

In [32]:
recent = df[df['Year'] >= 1994].sort_values(by="Year").reset_index(drop=True)
past  = df[df['Year'] < 1994].sort_values(by="Year").reset_index(drop=True)

In [33]:
recent.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Battle Arena Toshinden,PS,1994,Fighting,Sony Computer Entertainment,390000,260000,530000,80000
1,Yuu Yuu Hakusho: Makyo Toitsusen,GEN,1994,Fighting,Sega,0,0,80000,0
2,Super Famista 3,SNES,1994,Sports,Namco Bandai Games,0,0,390000,0
3,Jikkyou Powerful Pro Yakuu '94,SNES,1994,Sports,Konami Digital Entertainment,0,0,410000,0
4,NBA Jam,SNES,1994,Sports,Acclaim Entertainment,1190000,160000,0,30000


In [34]:
past.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Boxing,2600,1980,Fighting,Activision,720000,40000,0,10000
1,Freeway,2600,1980,Action,Activision,320000,20000,0,0
2,Defender,2600,1980,Misc,Atari,990000,50000,0,10000
3,Checkers,2600,1980,Misc,Atari,220000,10000,0,0
4,Missile Command,2600,1980,Shooter,Atari,2560000,170000,0,30000


In [35]:
recent.to_csv("data_94_21.csv", index=False)
past.to_csv("data_80_94.csv", index=False)

---
---